In [125]:
categories = ['Airport', 'City', 'Company', 'Compound', 'Conference', 'Lake', 'Person']
idx = 6

preds_path = f'../outputs/sample/{categories[idx]}.json'
baseline_path = f'../outputs/sample/el_baseline/{categories[idx]}.json'
# baseline_path = f'../outputs/sample/el_wo_exact/{categories[idx]}.json'
true_path = f'/data1/ujiie/shinra/EN/linkjp-sample-210402/link_annotation/{categories[idx]}.json'
tmp_path = '../outputs/tmp.json'

In [126]:
import json
base = []
with open(baseline_path, 'r') as f:
    for line in f:
        line = line.rstrip()
        if not line:
            continue
        line = json.loads(line)
        base.append(line)

print(len(base))
base_set = set([(p['page_id'], p['html_offset']['start']['line_id'], p['html_offset']['start']['offset']) for p in base])

1626


In [127]:
import json
trues = []
with open(true_path, 'r') as f:
    for line in f:
        line = line.rstrip()
        if not line:
            continue
        line = json.loads(line)
        trues.append(line)
print(len(trues))
true_set = set([(p['page_id'], p['html_offset']['start']['line_id'], p['html_offset']['start']['offset']) for p in trues])

1731


In [134]:
preds = []
with open(preds_path, 'r') as f:
    for line in f:
        line = line.rstrip()
        if not line:
            continue
        line = json.loads(line)
        preds.append(line)
print(len(preds))

3326


In [130]:
import sys
sys.path.append('/home/is/ujiie')
import linkjp_scorer
from linkjp_scorer.scorer.core import micro_average

thresholds = list(range(-20, 21))

max_thre = - float('inf')
max_f1 = - float('inf')
scores = None

for thre in thresholds:
    with open(tmp_path, 'w') as f:
        for b in base:
            b['page_id'] = str(b['page_id'])
            f.write(json.dumps(b, ensure_ascii=False) + '\n')

        for p in preds:
            p['page_id'] = str(p['page_id'])
            cand = (str(p['page_id']), p['html_offset']['start']['line_id'], p['html_offset']['start']['offset'])
            if cand in base_set:
                continue


            if p['score'] > thre:
                f.write(json.dumps(p, ensure_ascii=False) + '\n')

    scorer = linkjp_scorer.Scorer(linkjp_scorer.Category.PERSON,
                                    true_path, tmp_path)
    scorer.calc_score()
    s = micro_average(scorer.counter.values())
    f1 = s.f1
    if f1 > max_f1:
        max_f1 = f1
        max_thre = thre
        scores = (s.precision, s.recall, s.f1)
max_f1, max_thre, scores


(0.50238379022646,
 -20,
 (0.518450184501845, 0.48728323699421966, 0.50238379022646))

## bertのみ
- Airport 5, 0.658
- City 4, 0.700
- COMPANY 6, 0.557
- COMPOUND 4, 0.535
- CONFERENCE 3, 0.721
- LAKE 3, 0.733
- PERSON 6, 0.526

## baseline
- Airport null, 0.839
- City null, 0.682
- COMPANY null, 0.545
- COMPOUND null, 0.845
- CONFERENCE null, 0.713
- LAKE null, 0.761
- PERSON null, 0.502

## baseline + bert
- Airport 10, 0.852
- CITY 12, 0.684
- COMPANY 9, 0.574
- COMPOUND 14, 0.851
- CONFERENCE 12, 0.715
- LAKE 10, 0.766
- PERSON 9, 0.513

## baseline wo exact match
- Airport null, 0.558
- CITY null, 0.169
- COMPANY null, 0.201
- COMPOUND null, 0.657
- CONFERENCE null, 0.125
- LAKE null, 0.206
- PERSON null, 0.127

## baseline wo exact match + bert
- Airport 5, 0.855
- CITY 4, 0.721
- COMPANY 6, 0.583
- COMPOUND 9, 0.837
- CONFERENCE 4, 0.732
- LAKE 5, 0.777
- PERSON 6, 0.552

In [151]:
idx = 6
thre = 6

categories = ['Airport', 'City', 'Company', 'Compound', 'Conference', 'Lake', 'Person']
preds_path = f'../outputs/{categories[idx]}.json'
baseline_path = f'../outputs/el_wo_exact/{categories[idx]}.json'
output_path = f'../outputs/eval_only_bert/{categories[idx]}.json'


base = []
with open(baseline_path, 'r') as f:
    for line in f:
        line = line.rstrip()
        if not line:
            continue
        line = json.loads(line)
        base.append(line)
# base_set = set([(p['page_id'], p['html_offset']['start']['line_id'], p['html_offset']['start']['offset'], p['html_offset']['end']['offset']) for p in base])
base_set = set([(p['page_id'], p['html_offset']['start']['line_id'], p['html_offset']['start']['offset'], p['html_offset']['end']['offset']) for p in base])
print(len(base_set))

preds = []
with open(preds_path, 'r') as f:
    for line in f:
        line = line.rstrip()
        if not line:
            continue
        line = json.loads(line)
        preds.append(line)
print(len(preds))
cnt = 0

with open(output_path, 'w') as f:
    for b in base:
        b['page_id'] = str(b['page_id'])
        f.write(json.dumps(b, ensure_ascii=False) + '\n')

    for p in preds:
        p['page_id'] = str(p['page_id'])
        cand = (str(p['page_id']), p['html_offset']['start']['line_id'], p['html_offset']['start']['offset'], p['html_offset']['end']['offset'])
        if cand in base_set:
            cnt += 1
            continue


        if p['score'] > thre:
            del p['score']
            del p['candidates']
            f.write(json.dumps(p, ensure_ascii=False) + '\n')

print(cnt)

422
4026
449
